In [ ]:
!pip install scikit-learn pandas nltk

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
!pip install contractions
import re
import string
import contractions
import nltk
import gensim
from gensim.utils import simple_preprocess
import spacy
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 5.6 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# PS discussion

In [ ]:
ps = pd.read_csv('/content/dataset_reddit-scraper-lite-task_2024-08-26_07-52-20-549.csv')
ps.head()

,body,category,communityName,createdAt,dataType,flair,html,id,isAd,isVideo,...,parsedId,postId,scrapedAt,thumbnailUrl,title,upVoteRatio,upVotes,url,userId,username
0,So I had this epiphany the other day playing B...,NaN,r/PS5,2022-12-30T05:48:41.000Z,post,Discussion,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",t3_zysi03,False,False,...,zysi03,NaN,2024-08-26T07:49:47.105Z,self,The PS5 is the first console since PS2 that fe...,0.75,3560,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_6okblemr,MomentHead
1,I agree with you in principle but I think your...,PS5,r/PS5,2022-12-30T11:55:25.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;I agree with yo...",t1_j28geqi,NaN,NaN,...,t1_j28geqi,t3_zysi03,2024-08-26T07:49:55.352Z,NaN,NaN,NaN,2003,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_ekccm,hoochiscrazy_
2,I agree with you in principle but I think your...,PS5,r/PS5,2022-12-30T11:55:25.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;I agree with yo...",t1_j28geqi,NaN,NaN,...,t1_j28geqi,t3_zysi03,2024-08-26T07:49:50.751Z,NaN,NaN,NaN,2005,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_ekccm,hoochiscrazy_
3,And don't forget an actual UI for doing stuff ...,PS5,r/PS5,2022-12-30T13:36:21.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;And don&amp;#39...",t1_j28q47t,NaN,NaN,...,t1_j28q47t,t3_zysi03,2024-08-26T07:49:55.352Z,NaN,NaN,NaN,679,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_4bl7afza,xBesto
4,I agree with you in principle but I think your...,PS5,r/PS5,2022-12-30T11:55:25.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;I agree with yo...",t1_j28geqi,NaN,NaN,...,t1_j28geqi,t3_zysi03,2024-08-26T07:49:47.137Z,NaN,NaN,NaN,1998,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_ekccm,hoochiscrazy_


## NMF

In [ ]:
def clean_text_data(df, target_column):
    """
    Clean the text data in the specified column of the DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the text data.
    target_column (str): The name of the column to clean.

    Returns:
    pd.DataFrame: The cleaned DataFrame with the specified column preprocessed.
    """

    # Drop data with missing values in the target column
    df = df.dropna(subset=[target_column])

    # Drop duplicate review content
    df = df.drop_duplicates(subset=[target_column])

    # Convert the relevant column to lowercase
    df[target_column] = df[target_column].str.lower()

    # Remove contractions
    df[target_column] = df[target_column].map(lambda x: contractions.fix(x))

    # Remove invalid encoded characters (such as “�”)
    df[target_column] = df[target_column].map(lambda x: re.sub(r'[�]', '', x))

    # Remove websites (URLs)
    df[target_column] = df[target_column].map(lambda x: re.sub(r'http://\S+|https://\S+', '', x))

    # Remove emojis
    emoj = re.compile("["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                      u"\U00002500-\U00002BEF"  # chinese char
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"  # dingbats
                      u"\u3030"
                      "]+", re.UNICODE)
    df[target_column] = df[target_column].map(lambda x: re.sub(emoj, '', x))

    # Remove @ mentions
    df[target_column] = df[target_column].map(lambda x: re.sub(r'@\w+', '', x))

    # Remove hashtags
    df[target_column] = df[target_column].map(lambda x: re.sub(r'#\w+', '', x))

    # Remove non-word characters (e.g., numbers, special characters)
    df[target_column] = df[target_column].str.replace("[^A-Za-z]", " ", regex=True)

    # Remove extra spaces
    df[target_column] = df[target_column].map(lambda x: re.sub(r'\s{2,}', " ", x))

    # Remove punctuation
    df[target_column] = df[target_column].map(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))

    return df

In [ ]:
cleaned_ps = clean_text_data(ps, 'body')
cleaned_ps

,body,category,communityName,createdAt,dataType,flair,html,id,isAd,isVideo,...,parsedId,postId,scrapedAt,thumbnailUrl,title,upVoteRatio,upVotes,url,userId,username
0,so i had this epiphany the other day playing b...,NaN,r/PS5,2022-12-30T05:48:41.000Z,post,Discussion,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",t3_zysi03,False,False,...,zysi03,NaN,2024-08-26T07:49:47.105Z,self,The PS5 is the first console since PS2 that fe...,0.75,3560,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_6okblemr,MomentHead
1,i agree with you in principle but i think your...,PS5,r/PS5,2022-12-30T11:55:25.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;I agree with yo...",t1_j28geqi,NaN,NaN,...,t1_j28geqi,t3_zysi03,2024-08-26T07:49:55.352Z,NaN,NaN,NaN,2003,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_ekccm,hoochiscrazy_
3,and do not forget an actual ui for doing stuff...,PS5,r/PS5,2022-12-30T13:36:21.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;And don&amp;#39...",t1_j28q47t,NaN,NaN,...,t1_j28q47t,t3_zysi03,2024-08-26T07:49:55.352Z,NaN,NaN,NaN,679,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_4bl7afza,xBesto
7,was big into torrenting movies back then being...,PS5,r/PS5,2022-12-30T13:52:23.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;Was big into to...",t1_j28rywp,NaN,NaN,...,t1_j28rywp,t3_zysi03,2024-08-26T07:49:55.352Z,NaN,NaN,NaN,223,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_7hp0jydd,No-Release-6464
10,same here,PS5,r/PS5,2022-12-30T13:56:50.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;100% Same here&...",t1_j28si8h,NaN,NaN,...,t1_j28si8h,t3_zysi03,2024-08-26T07:49:55.352Z,NaN,NaN,NaN,42,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_ekccm,hoochiscrazy_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,your pc cannot play a game browse the web and ...,PS5,r/PS5,2022-12-30T14:10:53.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;Your pc can’t p...",t1_j28u8nn,NaN,NaN,...,t1_j28u8nn,t3_zysi03,2024-08-26T07:49:55.356Z,NaN,NaN,NaN,7,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_11t6b7,DiggingPodcast
641,you need to upgrade your pc,PS5,r/PS5,2022-12-30T06:58:59.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;you need to upg...",t1_j27to92,NaN,NaN,...,t1_j27to92,t3_zysi03,2024-08-26T07:49:55.356Z,NaN,NaN,NaN,10,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_tzbguusk,trowin_away
647,i do not know all that feel next gen about the...,PS5,r/PS5,2022-12-30T07:54:46.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;Idk all that fe...",t1_j27y815,NaN,NaN,...,t1_j27y815,t3_zysi03,2024-08-26T07:49:55.356Z,NaN,NaN,NaN,3,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_772g9,bulljoker
651,i have to admit that i enjoy the gap between p...,PS5,r/PS5,2022-12-30T11:52:02.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;I have to admit...",t1_j28g4k7,NaN,NaN,...,t1_j28g4k7,t3_zysi03,2024-08-26T07:49:55.356Z,NaN,NaN,NaN,3,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_wpe8u,Dycoth


In [ ]:
!pip install symspellpy
from symspellpy import SymSpell, Verbosity

# Initialize SymSpell object
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Load the dictionary file
dictionary_path = "/content/frequency_dictionary_en_82_765.txt"
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Optionally, you can load a bigram dictionary for phrase correction
bigram_path = "/content/frequency_bigramdictionary_en_243_342.txt"
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=1)


2024-08-28 16:34:41,841: E symspellpy.symspellpy] Dictionary file not found at /content/frequency_dictionary_en_82_765.txt.
ERROR:symspellpy.symspellpy:Dictionary file not found at /content/frequency_dictionary_en_82_765.txt.
2024-08-28 16:34:41,846: E symspellpy.symspellpy] Bigram dictionary file not found at /content/frequency_bigramdictionary_en_243_342.txt.
ERROR:symspellpy.symspellpy:Bigram dictionary file not found at /content/frequency_bigramdictionary_en_243_342.txt.


False

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models.phrases import Phrases, Phraser
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(df, target_column):
    # Step 1: Tokenize the words
    words = df[target_column].tolist()
    word_tokens = [word_tokenize(review) for review in words]

    # Step 2: Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized = []
    for review in word_tokens:
        n_lemmatized = [lemmatizer.lemmatize(w) for w in review]
        v_lemmatized = [lemmatizer.lemmatize(w, 'v') for w in n_lemmatized]
        r_lemmatized = [lemmatizer.lemmatize(w, 'r') for w in v_lemmatized]
        a_lemmatized = [lemmatizer.lemmatize(w, 'a') for w in r_lemmatized]
        lemmatized.append(a_lemmatized)

    # Step 3: Remove stopwords
    stop_words = set(stopwords.words("english"))
    # Add custom social media language words
    custom_stop_words = {"fuck", "bitch", "shit", "damn", "asshole"}
    stop_words.update(custom_stop_words)
    tokens_without_stopwords = [
        [w for w in review if w.lower() not in stop_words]
        for review in lemmatized
    ]

    # Step 4: Spell correction using SymSpell with both unigram and bigram dictionaries
    tokens_corrected = []
    for review in tokens_without_stopwords:
        corrected_review = []
        for word in review:
            suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
            corrected_word = suggestions[0].term if suggestions else word
            corrected_review.append(corrected_word)

        # Correct phrases using the bigram dictionary
        corrected_review = sym_spell.lookup_compound(" ".join(corrected_review), max_edit_distance=2)
        tokens_corrected.append(corrected_review[0].term.split())

    # Step 5: Create bigram and trigram models
    bigram = Phrases(tokens_corrected, min_count=3, threshold=10)
    trigram = Phrases(bigram[tokens_corrected], min_count=3, threshold=10)
    bigram_mod = Phraser(bigram)
    trigram_mod = Phraser(trigram)

    # Step 6: Form Bigrams and Trigrams
    tokens_with_bigrams = [bigram_mod[review] for review in tokens_corrected]
    tokens_with_trigrams = [trigram_mod[bigram_mod[review]] for review in tokens_corrected]

    # Step 7: Rejoin tokens into strings
    rejoin = []
    for review in tokens_with_trigrams:
        x = " ".join(review)  # Join tokens back into a single string
        x = x.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
        rejoin.append(x)

    # Step 8: Add the processed text back to the DataFrame
    df['filtered_text'] = rejoin

    return df


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
processed_ps = preprocess_text(cleaned_ps, 'body')
processed_ps.head()

,body,category,communityName,createdAt,dataType,flair,html,id,isAd,isVideo,...,postId,scrapedAt,thumbnailUrl,title,upVoteRatio,upVotes,url,userId,username,filtered_text
0,so i had this epiphany the other day playing b...,NaN,r/PS5,2022-12-30T05:48:41.000Z,post,Discussion,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",t3_zysi03,False,False,...,NaN,2024-08-26T07:49:47.105Z,self,The PS5 is the first console since PS2 that fe...,0.75,3560,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_6okblemr,MomentHead,epiphany day play biomutant game wa get butter...
1,i agree with you in principle but i think your...,PS5,r/PS5,2022-12-30T11:55:25.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;I agree with yo...",t1_j28geqi,NaN,NaN,...,t3_zysi03,2024-08-26T07:49:55.352Z,NaN,NaN,NaN,2003,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_ekccm,hoochiscrazy_,agree principle think point apply p personally...
3,and do not forget an actual ui for doing stuff...,PS5,r/PS5,2022-12-30T13:36:21.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;And don&amp;#39...",t1_j28q47t,NaN,NaN,...,t3_zysi03,2024-08-26T07:49:55.352Z,NaN,NaN,NaN,679,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_4bl7afza,xBesto,forget actual ui stuff besides look memory car...
7,was big into torrenting movies back then being...,PS5,r/PS5,2022-12-30T13:52:23.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;Was big into to...",t1_j28rywp,NaN,NaN,...,t3_zysi03,2024-08-26T07:49:55.352Z,NaN,NaN,NaN,223,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_7hp0jydd,No-Release-6464,wa big torrenting movie back able play memory ...
10,same here,PS5,r/PS5,2022-12-30T13:56:50.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;100% Same here&...",t1_j28si8h,NaN,NaN,...,t3_zysi03,2024-08-26T07:49:55.352Z,NaN,NaN,NaN,42,https://www.reddit.com/r/PS5/comments/zysi03/t...,t2_ekccm,hoochiscrazy_,


In [ ]:
vectorizer = TfidfVectorizer(max_features=10000)
tfidf_matrix = vectorizer.fit_transform(processed_ps['filtered_text'])

In [ ]:
nmf_model = NMF(n_components=6, init='nndsvd', random_state=42)
nmf_topics = nmf_model.fit_transform(tfidf_matrix)

for index, topic in enumerate(nmf_model.components_):
    print(f"Top 10 words for topic #{index}")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print("\n")

Top 10 words for topic #0
['player', 'launch', 'go', 'able', 'hd', 'time', 'leap', 'big', 'jump', 'wa']


Top 10 words for topic #1
['great', 'mean', 'thing', 'upgrade', 'literally', 'read', 'need', 'console', 'really', 'pc']


Top 10 words for topic #2
['almost', 'tough', 'sixaxis', 'battery', 'garbage', 'probably', 'also', 'favorite', 'use', 'controller']


Top 10 words for topic #3
['ha', 'like', 'time', 'still', 'play', 'really', 'well', 'fps', 'get', 'game']


Top 10 words for topic #4
['wii', 'generation', 'worth', 'definitely', 'loadtime', 'thing', 'feltlike', 'yup', 'feel', 'nextgen']


Top 10 words for topic #5
['alttab', 'fast', 'first', 'play', 'session', 'know', 'long', 'loadtime', 'game', 'lol']




## BERTopic

In [ ]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
from bertopic import BERTopic
import random
import numpy as np
import umap
from hdbscan import HDBSCAN

In [ ]:
random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

In [ ]:
umap_model = umap.UMAP(random_state=random_seed)
hdbscan_model = HDBSCAN()

In [ ]:
# Create BERTopic model
topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model)

ps = ps.drop_duplicates(subset=['body'])

# Fit and transform the documents
topics, probs = topic_model.fit_transform(ps['body'])

# Display topics
print(topic_model.get_topic_info())


   Topic  Count                           Name  \
0     -1      3            -1_ive_5th_mood_win   
1      0    148               0_the_and_to_was   
2      1     33                  1_pc_do_to_my   
3      2     12  2_controller_the_battery_just   

                                      Representation  \
0  [ive, 5th, mood, win, ds4, lose, you, owned, n...   
1    [the, and, to, was, of, it, that, ps3, in, ps5]   
2   [pc, do, to, my, on, that, game, can, cant, the]   
3  [controller, the, battery, just, is, controlle...   

                                 Representative_Docs  
0  [You win until you lose, they say., Really? Du...  
1  [PS1 to PS2 was also mindblowing back then. My...  
2  [My PC can do all of that, "throwing the conso...  
3  [The PS4 controller is probably my favorite PS...  


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ps_topic_info.csv", index=False)

In [ ]:
document_info = topic_model.get_document_info(ps['body'])
document_info.to_csv("ps_document_info.csv", index=False)

In [ ]:
ps.to_csv("ps.csv", index=False)

In [ ]:
import pandas as pd
ps_topic0_positive = pd.read_csv('/content/ps_topic0_positive.csv', encoding='ISO-8859-1')
ps_topic0_positive.head()

,Document
0,I agree with you in principle but I think your...
1,Was big into torrenting movies back then. Bei...
2,This. DNLA was a gamechanger for me.Combined w...
3,Ps3 menu is still my favourite console menu
4,Great point


In [ ]:
# Create BERTopic model
topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model)

# Fit and transform the documents
topics, probs = topic_model.fit_transform(ps_topic0_positive['Document'])

# Display topics
print(topic_model.get_topic_info())


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

   Topic  Count               Name  \
0     -1     29  -1_the_was_to_and   
1      0     29    0_the_and_to_of   
2      1     11    1_it_the_was_to   

                                      Representation  \
0     [the, was, to, and, ps3, it, of, ps5, me, for]   
1  [the, and, to, of, ps5, but, was, with, games,...   
2  [it, the, was, to, and, ps3, this, games, had,...   

                                 Representative_Docs  
0  [I agree 100%, the PS2 -&gt; PS3 jump blew my ...  
1  [I play a lot of older games and I agree. Swit...  
2  [This. DNLA was a gamechanger for me.Combined ...  


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ps_topic0_positive_info.csv", index=False)

In [ ]:
document_info = topic_model.get_document_info(ps_topic0_positive['Document'])
document_info.to_csv("ps_topic0_positive_document_info.csv", index=False)

In [ ]:
ps_topic0_negative = pd.read_csv('/content/ps_topic0_negative.csv', encoding='ISO-8859-1')
ps_topic0_negative.head()

,Document
0,"It did, but God it was janky."
1,I used to have the worst internet connection b...
2,Dreamcast really didn't deserve to be Sega's l...
3,"Completely agree, bad take by OP.\n\nPS2 was s..."
4,I hate the take that the PS3 was too expensive...


In [ ]:
topics, probs = topic_model.fit_transform(ps_topic0_negative['Document'])

print(topic_model.get_topic_info())

   Topic  Count                Name  \
0     -1     29    -1_the_and_it_to   
1      0      7  0_times_the_was_it   
2      1      6     1_the_to_and_of   

                                      Representation  \
0   [the, and, it, to, that, was, but, is, of, this]   
1  [times, the, was, it, load, bad, loading, when...   
2  [the, to, and, of, was, at, in, games, console...   

                                 Representative_Docs  
0  [And then there was the Wii. \n\nLike, graphic...  
1  [I forgot how bad it was until I was watching ...  
2  [PS5 is a great console but I disagree that it...  


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ps_topic0_negative_info.csv", index=False)

In [ ]:
document_info = topic_model.get_document_info(ps_topic0_negative['Document'])
document_info.to_csv("ps_topic0_negative_document_info.csv", index=False)

# Switch discussion

In [ ]:
ns = pd.read_csv('/content/dataset_reddit-scraper-lite-task_2024-08-28_12-18-07-772.csv')
ns.head()

,body,category,communityName,createdAt,dataType,flair,html,id,isAd,isVideo,...,parsedId,postId,scrapedAt,thumbnailUrl,title,upVoteRatio,upVotes,url,userId,username
0,The switch concept is genius and Nintendo need...,NaN,r/NintendoSwitch,2020-01-03T03:18:13.000Z,post,Discussion,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",t3_ej9c4w,False,False,...,ej9c4w,NaN,2024-08-28T12:16:51.006Z,self,Switch should be Nintendo's only console conce...,0.86,13331,https://www.reddit.com/r/NintendoSwitch/commen...,t2_30qlmm72,baddieuniverse
1,[deleted],NintendoSwitch,r/NintendoSwitch,2020-01-03T03:49:45.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;[deleted]&lt;/p...",t1_fcwe7u4,NaN,NaN,...,t1_fcwe7u4,t3_ej9c4w,2024-08-28T12:16:51.035Z,NaN,NaN,NaN,5909,https://www.reddit.com/r/NintendoSwitch/commen...,NaN,[deleted]
2,New Switch U Deluxe XL,NintendoSwitch,r/NintendoSwitch,2020-01-03T03:57:51.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;New Switch U De...",t1_fcwewue,NaN,NaN,...,t1_fcwewue,t3_ej9c4w,2024-08-28T12:16:51.035Z,NaN,NaN,NaN,2759,https://www.reddit.com/r/NintendoSwitch/commen...,t2_3qkrw,imyxle
3,Featuring Dante from the Devil May Cry series ...,NintendoSwitch,r/NintendoSwitch,2020-01-03T06:27:40.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;Featuring Dante...",t1_fcwppq3,NaN,NaN,...,t1_fcwppq3,t3_ej9c4w,2024-08-28T12:16:51.035Z,NaN,NaN,NaN,1805,https://www.reddit.com/r/NintendoSwitch/commen...,t2_k7txc,Blaine_Cooper
4,At the Olympics,NintendoSwitch,r/NintendoSwitch,2020-01-03T06:28:27.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;At the Olympics...",t1_fcwprg5,NaN,NaN,...,t1_fcwprg5,t3_ej9c4w,2024-08-28T12:16:51.035Z,NaN,NaN,NaN,761,https://www.reddit.com/r/NintendoSwitch/commen...,t2_dn86l,Alarid


## NMF

In [ ]:
cleaned_ns = clean_text_data(ns, 'body')
cleaned_ns

,body,category,communityName,createdAt,dataType,flair,html,id,isAd,isVideo,...,parsedId,postId,scrapedAt,thumbnailUrl,title,upVoteRatio,upVotes,url,userId,username
0,the switch concept is genius and nintendo need...,NaN,r/NintendoSwitch,2020-01-03T03:18:13.000Z,post,Discussion,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",t3_ej9c4w,False,False,...,ej9c4w,NaN,2024-08-28T12:16:51.006Z,self,Switch should be Nintendo's only console conce...,0.86,13331,https://www.reddit.com/r/NintendoSwitch/commen...,t2_30qlmm72,baddieuniverse
1,deleted,NintendoSwitch,r/NintendoSwitch,2020-01-03T03:49:45.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;[deleted]&lt;/p...",t1_fcwe7u4,NaN,NaN,...,t1_fcwe7u4,t3_ej9c4w,2024-08-28T12:16:51.035Z,NaN,NaN,NaN,5909,https://www.reddit.com/r/NintendoSwitch/commen...,NaN,[deleted]
2,new switch you deluxe xl,NintendoSwitch,r/NintendoSwitch,2020-01-03T03:57:51.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;New Switch U De...",t1_fcwewue,NaN,NaN,...,t1_fcwewue,t3_ej9c4w,2024-08-28T12:16:51.035Z,NaN,NaN,NaN,2759,https://www.reddit.com/r/NintendoSwitch/commen...,t2_3qkrw,imyxle
3,featuring dante from the devil may cry series ...,NintendoSwitch,r/NintendoSwitch,2020-01-03T06:27:40.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;Featuring Dante...",t1_fcwppq3,NaN,NaN,...,t1_fcwppq3,t3_ej9c4w,2024-08-28T12:16:51.035Z,NaN,NaN,NaN,1805,https://www.reddit.com/r/NintendoSwitch/commen...,t2_k7txc,Blaine_Cooper
4,at the olympics,NintendoSwitch,r/NintendoSwitch,2020-01-03T06:28:27.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;At the Olympics...",t1_fcwprg5,NaN,NaN,...,t1_fcwprg5,t3_ej9c4w,2024-08-28T12:16:51.035Z,NaN,NaN,NaN,761,https://www.reddit.com/r/NintendoSwitch/commen...,t2_dn86l,Alarid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,innovation comes with a cost but the pay off i...,NintendoSwitch,r/NintendoSwitch,2020-01-03T16:32:49.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;Innovation come...",t1_fcxqphf,NaN,NaN,...,t1_fcxqphf,t3_ej9c4w,2024-08-28T12:16:51.037Z,NaN,NaN,NaN,4,https://www.reddit.com/r/NintendoSwitch/commen...,t2_2bq6m1ob,Metashoby
434,the only innovation they need to worry about i...,NintendoSwitch,r/NintendoSwitch,2020-01-03T17:49:07.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;the only innova...",t1_fcy24qn,NaN,NaN,...,t1_fcy24qn,t3_ej9c4w,2024-08-28T12:16:51.037Z,NaN,NaN,NaN,3,https://www.reddit.com/r/NintendoSwitch/commen...,NaN,[deleted]
436,fix drift permanently,NintendoSwitch,r/NintendoSwitch,2020-01-03T23:06:43.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;Fix.&lt;/p&gt;\...",t1_fcz3axl,NaN,NaN,...,t1_fcz3axl,t3_ej9c4w,2024-08-28T12:16:51.037Z,NaN,NaN,NaN,4,https://www.reddit.com/r/NintendoSwitch/commen...,t2_xl6jh,DBMods
439,ah yes let us stop the company that thrives on...,NintendoSwitch,r/NintendoSwitch,2020-01-03T09:45:02.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;Ah yes, let&amp...",t1_fcwzpy6,NaN,NaN,...,t1_fcwzpy6,t3_ej9c4w,2024-08-28T12:16:51.037Z,NaN,NaN,NaN,16,https://www.reddit.com/r/NintendoSwitch/commen...,t2_kehchin,SteveTV1234


In [ ]:
processed_ns = preprocess_text(cleaned_ns, 'body')
processed_ns.head()

,body,category,communityName,createdAt,dataType,flair,html,id,isAd,isVideo,...,postId,scrapedAt,thumbnailUrl,title,upVoteRatio,upVotes,url,userId,username,filtered_text
0,the switch concept is genius and nintendo need...,NaN,r/NintendoSwitch,2020-01-03T03:18:13.000Z,post,Discussion,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",t3_ej9c4w,False,False,...,NaN,2024-08-28T12:16:51.006Z,self,Switch should be Nintendo's only console conce...,0.86,13331,https://www.reddit.com/r/NintendoSwitch/commen...,t2_30qlmm72,baddieuniverse,switch concept genius nintendo need build upon...
1,deleted,NintendoSwitch,r/NintendoSwitch,2020-01-03T03:49:45.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;[deleted]&lt;/p...",t1_fcwe7u4,NaN,NaN,...,t3_ej9c4w,2024-08-28T12:16:51.035Z,NaN,NaN,NaN,5909,https://www.reddit.com/r/NintendoSwitch/commen...,NaN,[deleted],delete
2,new switch you deluxe xl,NintendoSwitch,r/NintendoSwitch,2020-01-03T03:57:51.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;New Switch U De...",t1_fcwewue,NaN,NaN,...,t3_ej9c4w,2024-08-28T12:16:51.035Z,NaN,NaN,NaN,2759,https://www.reddit.com/r/NintendoSwitch/commen...,t2_3qkrw,imyxle,new switch deluxe xl
3,featuring dante from the devil may cry series ...,NintendoSwitch,r/NintendoSwitch,2020-01-03T06:27:40.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;Featuring Dante...",t1_fcwppq3,NaN,NaN,...,t3_ej9c4w,2024-08-28T12:16:51.035Z,NaN,NaN,NaN,1805,https://www.reddit.com/r/NintendoSwitch/commen...,t2_k7txc,Blaine_Cooper,feature dante devil may cry series amp knuckle
4,at the olympics,NintendoSwitch,r/NintendoSwitch,2020-01-03T06:28:27.000Z,comment,NaN,"&lt;div class=""md""&gt;&lt;p&gt;At the Olympics...",t1_fcwprg5,NaN,NaN,...,t3_ej9c4w,2024-08-28T12:16:51.035Z,NaN,NaN,NaN,761,https://www.reddit.com/r/NintendoSwitch/commen...,t2_dn86l,Alarid,olympics


In [ ]:
vectorizer = TfidfVectorizer(max_features=10000)
tfidf_matrix = vectorizer.fit_transform(processed_ns['filtered_text'])

In [ ]:
nmf_model = NMF(n_components=6, init='nndsvd', random_state=42)
nmf_topics = nmf_model.fit_transform(tfidf_matrix)

for index, topic in enumerate(nmf_model.components_):
    print(f"Top 10 words for topic #{index}")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print("\n")

Top 10 words for topic #0
['game', 'new', 'beauty', 'right', 'cube', 'nintendo', 'mii', 'blade', 'uwu', 'switch']


Top 10 words for topic #1
['could', 'would', 'think', 'next', 'get', 'game', 'keep', 'nintendo', 'console', 'thing']


Top 10 words for topic #2
['dante', 'knuckle', 'ultra', 'mode', 'funky', 'hold', 'deluxe', 'series', 'xl', 'new']


Top 10 words for topic #3
['also', 'new', 'something', 'rename', 'game', 'though', 'letu', 'swii', 'nintendo', 'swiitch']


Top 10 words for topic #4
['system', 'switchcube', 'name', 'get', 'boy', 'snes', 'sn', 'nintendo', 'entertainment', 'super']


Top 10 words for topic #5
['play', 'use', 'name', 'would', 'one', 'game', 'controller', 'wii', 'screen', 'wa']




## BERTopic

In [ ]:
ns = ns.drop_duplicates(subset=['body'])

topics, probs = topic_model.fit_transform(ns['body'])

print(topic_model.get_topic_info())

   Topic  Count                       Name  \
0     -1     33          -1_the_of_that_to   
1      0     79            0_the_to_and_it   
2      1     74  1_switch_the_new_nintendo   

                                      Representation  \
0    [the, of, that, to, it, and, is, as, this, one]   
1  [the, to, and, it, is, with, that, of, they, for]   
2  [switch, the, new, nintendo, super, xl, name, ...   

                                 Representative_Docs  
0  [I'm an audio student as well and part of it i...  
1  [I find these kinds of statements a bit iffy b...  
2  [There's nothing wrong with Switch 2 just like...  


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ns_topic_info.csv", index=False)

In [ ]:
document_info = topic_model.get_document_info(ns['body'])
document_info.to_csv("ns_document_info.csv", index=False)

# PS5

In [ ]:
ps5_launch = pd.read_csv('/content/ps5 launch.csv')
ps5_launch.head()

,url,twitterUrl,id,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote
0,https://x.com/B_Kaws/status/1810019659233828941,https://twitter.com/B_Kaws/status/181001965923...,1810019659233828941,@Rosethesmol @PlayStation @AskPlayStation Maybe,0,0,0,0,Sun Jul 07 18:34:49 +0000 2024,0,False,False
1,https://x.com/Blungusfart/status/1434317891306...,https://twitter.com/Blungusfart/status/1434317...,1434317891306917892,@XPraetor @Mjolnir_1 @ZailmanY @Jmlluna23b @Pl...,0,0,0,0,Sun Sep 05 00:50:08 +0000 2021,0,False,False
2,https://x.com/Dbob93/status/1404422373969956865,https://twitter.com/Dbob93/status/140442237396...,1404422373969956865,@PlayStation Y’all it’s been a year and i stil...,0,0,0,0,Mon Jun 14 12:56:01 +0000 2021,0,False,False
3,https://x.com/Peyton45359569/status/1346281829...,https://twitter.com/Peyton45359569/status/1346...,1346281829146398720,"@AK74785811 @DrGuru23 @PlayStation ""Baddest"" l...",0,0,0,0,Tue Jan 05 02:26:15 +0000 2021,0,False,False
4,https://x.com/liv_mae69/status/131893921240259...,https://twitter.com/liv_mae69/status/131893921...,1318939212402597888,@dion_roberts23 @PlayStation @PlayStationUK iv...,0,0,0,0,Wed Oct 21 15:36:27 +0000 2020,0,False,False


## NMF

In [ ]:
def clean_text_data(df, target_column):
    """
    Clean the text data in the specified column of the DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the text data.
    target_column (str): The name of the column to clean.

    Returns:
    pd.DataFrame: The cleaned DataFrame with the specified column preprocessed.
    """

    # Drop data with missing values in the target column
    df = df.dropna(subset=[target_column])

    # Drop duplicate review content
    df = df.drop_duplicates(subset=[target_column])

    # Convert the relevant column to lowercase
    df[target_column] = df[target_column].str.lower()

    # Remove contractions
    df[target_column] = df[target_column].map(lambda x: contractions.fix(x))

    # Remove invalid encoded characters (such as “�”)
    df[target_column] = df[target_column].map(lambda x: re.sub(r'[�]', '', x))

    # Remove websites (URLs)
    df[target_column] = df[target_column].map(lambda x: re.sub(r'http://\S+|https://\S+', '', x))

    # Remove emojis
    emoj = re.compile("["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                      u"\U00002500-\U00002BEF"  # chinese char
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"  # dingbats
                      u"\u3030"
                      "]+", re.UNICODE)
    df[target_column] = df[target_column].map(lambda x: re.sub(emoj, '', x))

    # Remove @ mentions
    df[target_column] = df[target_column].map(lambda x: re.sub(r'@\w+', '', x))

    # Remove hashtags
    df[target_column] = df[target_column].map(lambda x: re.sub(r'#\w+', '', x))

    # Remove non-word characters (e.g., numbers, special characters)
    df[target_column] = df[target_column].str.replace("[^A-Za-z]", " ", regex=True)

    # Remove extra spaces
    df[target_column] = df[target_column].map(lambda x: re.sub(r'\s{2,}', " ", x))

    # Remove punctuation
    df[target_column] = df[target_column].map(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x))

    return df

In [ ]:
cleaned_ps5 = clean_text_data(ps5_launch, 'text')
cleaned_ps5.head()

,url,twitterUrl,id,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote
0,https://x.com/B_Kaws/status/1810019659233828941,https://twitter.com/B_Kaws/status/181001965923...,1810019659233828941,maybe,0,0,0,0,Sun Jul 07 18:34:49 +0000 2024,0,False,False
1,https://x.com/Blungusfart/status/1434317891306...,https://twitter.com/Blungusfart/status/1434317...,1434317891306917892,ratio,0,0,0,0,Sun Sep 05 00:50:08 +0000 2021,0,False,False
2,https://x.com/Dbob93/status/1404422373969956865,https://twitter.com/Dbob93/status/140442237396...,1404422373969956865,you all it is been a year and i still cannot ...,0,0,0,0,Mon Jun 14 12:56:01 +0000 2021,0,False,False
3,https://x.com/Peyton45359569/status/1346281829...,https://twitter.com/Peyton45359569/status/1346...,1346281829146398720,baddest lmfao,0,0,0,0,Tue Jan 05 02:26:15 +0000 2021,0,False,False
4,https://x.com/liv_mae69/status/131893921240259...,https://twitter.com/liv_mae69/status/131893921...,1318939212402597888,iv never met anyone so stupid in my life,0,0,0,0,Wed Oct 21 15:36:27 +0000 2020,0,False,False


In [ ]:
!pip install symspellpy
from symspellpy import SymSpell, Verbosity

# Initialize SymSpell object
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Load the dictionary file
dictionary_path = "/content/frequency_dictionary_en_82_765.txt"
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Optionally, you can load a bigram dictionary for phrase correction
bigram_path = "/content/frequency_bigramdictionary_en_243_342.txt"
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=1)


True

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models.phrases import Phrases, Phraser
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(df, target_column):
    # Step 1: Tokenize the words
    words = df[target_column].tolist()
    word_tokens = [word_tokenize(review) for review in words]

    # Step 2: Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatised = []
    for review in word_tokens:
        n_lemmatised = [lemmatizer.lemmatize(w) for w in review]
        v_lemmatised = [lemmatizer.lemmatize(w, 'v') for w in n_lemmatised]
        r_lemmatised = [lemmatizer.lemmatize(w, 'r') for w in v_lemmatised]
        a_lemmatised = [lemmatizer.lemmatize(w, 'a') for w in r_lemmatised]
        lemmatised.append(a_lemmatised)

    # Step 3: Remove stopwords
    stop_words = set(stopwords.words("english"))
    # Add custom social media language words
    custom_stop_words = {"fuck", "bitch", "shit", "damn", "asshole"}
    stop_words.update(custom_stop_words)
    tokens_without_stopwords = [
        [w for w in review if w.lower() not in stop_words]
        for review in lemmatised
    ]

    # Step 4: Spell correction using SymSpell with both unigram and bigram dictionaries
    tokens_corrected = []
    for review in tokens_without_stopwords:
        corrected_review = []
        for word in review:
            suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
            corrected_word = suggestions[0].term if suggestions else word
            corrected_review.append(corrected_word)

        # Correct phrases using the bigram dictionary
        corrected_review = sym_spell.lookup_compound(" ".join(corrected_review), max_edit_distance=2)
        tokens_corrected.append(corrected_review[0].term.split())

    # Step 5: Create bigram and trigram models
    bigram = Phrases(tokens_corrected, min_count=3, threshold=10)
    trigram = Phrases(bigram[tokens_corrected], min_count=3, threshold=10)
    bigram_mod = Phraser(bigram)
    trigram_mod = Phraser(trigram)

    # Step 6: Form Bigrams and Trigrams
    tokens_with_bigrams = [bigram_mod[review] for review in tokens_corrected]
    tokens_with_trigrams = [trigram_mod[bigram_mod[review]] for review in tokens_corrected]

    # Step 7: Rejoin tokens into strings
    rejoin = []
    for review in tokens_with_trigrams:
        x = " ".join(review)  # Join tokens back into a single string
        x = x.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
        rejoin.append(x)

    # Step 8: Add the processed text back to the DataFrame
    df['filtered_text'] = rejoin

    return df


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
processed_ps5 = preprocess_text(cleaned_ps5, 'text')
processed_ps5.head()

,url,twitterUrl,id,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote,filtered_text
0,https://x.com/B_Kaws/status/1810019659233828941,https://twitter.com/B_Kaws/status/181001965923...,1810019659233828941,maybe,0,0,0,0,Sun Jul 07 18:34:49 +0000 2024,0,False,False,maybe
1,https://x.com/Blungusfart/status/1434317891306...,https://twitter.com/Blungusfart/status/1434317...,1434317891306917892,ratio,0,0,0,0,Sun Sep 05 00:50:08 +0000 2021,0,False,False,ratio
2,https://x.com/Dbob93/status/1404422373969956865,https://twitter.com/Dbob93/status/140442237396...,1404422373969956865,you all it is been a year and i still cannot ...,0,0,0,0,Mon Jun 14 12:56:01 +0000 2021,0,False,False,year still get one of join
3,https://x.com/Peyton45359569/status/1346281829...,https://twitter.com/Peyton45359569/status/1346...,1346281829146398720,baddest lmfao,0,0,0,0,Tue Jan 05 02:26:15 +0000 2021,0,False,False,baddest lao
4,https://x.com/liv_mae69/status/131893921240259...,https://twitter.com/liv_mae69/status/131893921...,1318939212402597888,iv never met anyone so stupid in my life,0,0,0,0,Wed Oct 21 15:36:27 +0000 2020,0,False,False,in never meet anyone stupid life


In [ ]:
vectorizer = TfidfVectorizer(max_features=10000)
tfidf_matrix = vectorizer.fit_transform(processed_ps5['filtered_text'])

In [ ]:
nmf_model = NMF(n_components=5, init='nndsvd', random_state=42)
nmf_topics = nmf_model.fit_transform(tfidf_matrix)

for index, topic in enumerate(nmf_model.components_):
    print(f"Top 10 words for topic #{index}")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print("\n")

Top 10 words for topic #0
['playstation', 'design', 'looklike', 'go', 'one', 'console', 'game', 'get', 'like', 'box']


Top 10 words for topic #1
['areorder', 'go', 'drop', 'releasedate', 'want', 'know', 'though', 'please', 'need', 'price']


Top 10 words for topic #2
['see', 'one', 'make', 'need', 'controller', 'get', 'doecome', 'white', 'please', 'black']


Top 10 words for topic #3
['white', 'design', 'clean', 'bad', 'cool', 'amaze', 'good', 'nice', 'well', 'look']


Top 10 words for topic #4
['thank', 'want', 'know', 'doecost', 'cool', 'cost', 'of', 'well', 'though', 'much']




## BERTopic

In [ ]:
!pip install bertopic

In [ ]:
from bertopic import BERTopic
import random
import numpy as np
import umap
from hdbscan import HDBSCAN

In [ ]:
random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

In [ ]:
umap_model = umap.UMAP(random_state=random_seed)
hdbscan_model = HDBSCAN()

In [ ]:
topic_model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model)

import re
# Remove websites (URLs)
ps5_launch['text'] = ps5_launch['text'].map(lambda x: re.sub(r'http://\S+|https://\S+', '', x))

# Remove @ mentions
ps5_launch['text'] = ps5_launch['text'].map(lambda x: re.sub(r'@\w+', '', x))

# Remove hashtags
ps5_launch['text'] = ps5_launch['text'].map(lambda x: re.sub(r'#\w+', '', x))


topics, probs = topic_model.fit_transform(ps5_launch['text'])
print(topic_model.get_topic_info())

     Topic  Count                                     Name  \
0       -1   1574                       -1_and_games_to_it   
1        0    302                       0_ps5_ps4_the_slim   
2        1     95             1_router_wifi_internet_looks   
3        2     93              2_design_change_love_notice   
4        3     67            3_console_consoles_ever_looks   
..     ...    ...                                      ...   
173    172      5                 172_giveaway_take_two_na   
174    173      5                   173_4k_100000th_por_hr   
175    174      5     174_theirs_ahh_surrounding_hypocrite   
176    175      5            175_side_99_rectangle_nowhere   
177    176      5  176_complexes_crossplatform_buds_atrted   

                                        Representation  \
0    [and, games, to, it, you, they, the, on, that,...   
1    [ps5, ps4, the, slim, ps3, is, playstation, of...   
2    [router, wifi, internet, looks, like, fancy, r...   
3    [design, change, l

In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ps5_launch_topic_info.csv", index=False)

In [ ]:
document_info = topic_model.get_document_info(ps5_launch['text'])
document_info.to_csv("ps5_launch_document_info.csv", index=False)

In [ ]:
ps5_launch_sentiment = pd.read_csv('/content/ps5_launch_sentiment.csv')
ps5_launch_topic = pd.read_csv('/content/ps5_launch_document_info (1).csv',encoding='ISO-8859-1')

In [ ]:
merged_ps5 = pd.merge(ps5_launch_topic, ps5_launch_sentiment, on='url', how='inner')
merged_ps5

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,url,twitterUrl,...,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote,Transformer_Sentiment
0,Maybe,112,112_nut_classs_roughly_maybe,"['nut', 'classs', 'roughly', 'maybe', 'later',...","["" maybe later don't know"", ' Top classs', ' ...",nut - classs - roughly - maybe - later - bit -...,1.000000,False,https://x.com/B_Kaws/status/1810019659233828941,https://twitter.com/B_Kaws/status/181001965923...,...,maybe,0,0,0,0,Sun Jul 07 18:34:49 +0000 2024,0,False,False,Neutral
1,Ratio,-1,-1_and_games_to_it,"['and', 'games', 'to', 'it', 'you', 'they', 't...",[' We buy these consoles to play games on them...,and - games - to - it - you - they - the - on ...,0.000000,False,https://x.com/Blungusfart/status/1434317891306...,https://twitter.com/Blungusfart/status/1434317...,...,ratio,0,0,0,0,Sun Sep 05 00:50:08 +0000 2021,0,False,False,Neutral
2,Y¡¯all it¡¯s been a year and i still can¡¯t g...,92,92_future_goin_now_truly,"['future', 'goin', 'now', 'truly', '2005', 're...","[' ""The Future is Now"" ', ' this is future!!'...",future - goin - now - truly - 2005 - remind - ...,0.353828,False,https://x.com/Dbob93/status/1404422373969956865,https://twitter.com/Dbob93/status/140442237396...,...,you all it is been a year and i still cannot ...,0,0,0,0,Mon Jun 14 12:56:01 +0000 2021,0,False,False,Negative
3,"""Baddest"" lmfao",65,65_baddest_fake_edit_real,"['baddest', 'fake', 'edit', 'real', 'bahhagaha...","[' Very fake. Not only a different font, the...",baddest - fake - edit - real - bahhagahahaha -...,1.000000,False,https://x.com/Peyton45359569/status/1346281829...,https://twitter.com/Peyton45359569/status/1346...,...,baddest lmfao,0,0,0,0,Tue Jan 05 02:26:15 +0000 2021,0,False,False,Negative
4,iv never met anyone so stupid in my life ????,113,113_smart_fun_yawning_crayon,"['smart', 'fun', 'yawning', 'crayon', 'loser',...","["" They're hidden under all the crayon eate...",smart - fun - yawning - crayon - loser - dumba...,1.000000,False,https://x.com/liv_mae69/status/131893921240259...,https://twitter.com/liv_mae69/status/131893921...,...,iv never met anyone so stupid in my life,0,0,0,0,Wed Oct 21 15:36:27 +0000 2020,0,False,False,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4943,Great! Now... Can we get one in black?,121,121_black_accessory_coool_pack,"['black', 'accessory', 'coool', 'pack', 'cacan...","[' Can i have it in black?', ' ca...can you ge...",black - accessory - coool - pack - cacan - mat...,1.000000,False,https://x.com/NoxRaijin/status/127119920726170...,https://twitter.com/NoxRaijin/status/127119920...,...,great now can we get one in black,0,0,0,0,Thu Jun 11 21:54:43 +0000 2020,0,False,False,Positive
4944,You sure got em,-1,-1_and_games_to_it,"['and', 'games', 'to', 'it', 'you', 'they', 't...",[' We buy these consoles to play games on them...,and - games - to - it - you - they - the - on ...,0.000000,False,https://x.com/Cat_In_Da__Box/status/1271199207...,https://twitter.com/Cat_In_Da__Box/status/1271...,...,you sure got them,0,1,1,0,Thu Jun 11 21:54:43 +0000 2020,0,False,False,Positive
4945,Probably what¡¯s they will be or 100 less each,-1,-1_and_games_to_it,"['and', 'games', 'to', 'it', 'you', 'they', 't...",[' We buy these consoles to play games on them...,and - games - to - it - you - they - the - on ...,0.000000,False,https://x.com/FireUpUmich/status/1271199204392...,https://twitter.com/FireUpUmich/status/1271199...,...,probably what is they will be or less each,0,0,2,0,Thu Jun 11 21:54:42 +0000 2020,0,False,False,Neutral
4946,At the end most of the people can't buy eve...,-1,-1_and_games_to_it,"['and', 'games', 'to', 'it', 'you', 'they', 't...",[' We buy these consoles to play games on them...,and - games - to - it - you - they - the - on ...,0.000000,False,https://x.com/anatoly_amzin/status/12711992023...,https://twitter.com/anatoly_am

In [ ]:
merged_ps5.to_csv("ps5_launch_topic_sentiment.csv",index=False)

In [ ]:
ps5_0_p = pd.read_csv('/content/ps5_0_p.csv')

In [ ]:
topics, probs = topic_model.fit_transform(ps5_0_p['Document_x'])
print(topic_model.get_topic_info())

   Topic  Count                          Name  \
0     -1      4  -1_to_my_playstation_edition   
1      0     61              0_the_ps5_to_and   
2      1     11            1_ps4_my_this_with   

                                      Representation  \
0  [to, my, playstation, edition, met, friends, b...   
1    [the, ps5, to, and, is, it, like, its, you, so]   
2  [ps4, my, this, with, it, the, make, but, not,...   

                                 Representative_Docs  
0  [  My pro does it :( the 500 million edition i...  
1  [ The ps5 looks like it's gonna be great and h...  
2  [ Well.. Not bad.. \n\nBut now.. I'll keep my ...  


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ps5_0_p_topic_info.csv", index=False)
document_info = topic_model.get_document_info(ps5_0_p['Document_x'])
document_info.to_csv("ps5_0_p_document_info.csv", index=False)

In [ ]:
ps5_0_n = pd.read_csv('/content/ps5_0_n.csv')

In [ ]:
topics, probs = topic_model.fit_transform(ps5_0_n['Document_x'])
print(topic_model.get_topic_info())

   Topic  Count                Name  \
0     -1     27  -1_the_is_ps5_that   
1      0     31    0_the_ps5_is_and   
2      1     14   1_the_ps5_that_to   
3      2      7   2_ps4_ago_got_now   

                                      Representation  \
0  [the, is, ps5, that, xbox, and, to, of, but, why]   
1  [the, ps5, is, and, design, you, like, to, of,...   
2  [the, ps5, that, to, not, be, dont, get, just,...   
3  [ps4, ago, got, now, it, only, my, all, out, w...   

                                 Representative_Docs  
0  [ Every playstation I've ever bought I've had ...  
1  [ Kind of looks like someone behind the PS5 as...  
2  [ Mennn ttsss whats the point of letting us se...  
3  [  only the standard PS4's do this lol i hate ...  


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ps5_0_n_topic_info.csv", index=False)
document_info = topic_model.get_document_info(ps5_0_n['Document_x'])
document_info.to_csv("ps5_0_n_document_info.csv", index=False)

In [ ]:
ps5_0_p = pd.read_csv('/content/ps5_0_p.csv')

In [ ]:
topics, probs = topic_model.fit_transform(ps5_0_p['Document_x'])
print(topic_model.get_topic_info())

   Topic  Count                          Name  \
0     -1      4  -1_to_my_playstation_edition   
1      0     61              0_the_ps5_to_and   
2      1     11            1_ps4_my_this_with   

                                      Representation  \
0  [to, my, playstation, edition, met, friends, b...   
1    [the, ps5, to, and, is, it, like, its, you, so]   
2  [ps4, my, this, with, it, the, make, but, not,...   

                                 Representative_Docs  
0  [  My pro does it :( the 500 million edition i...  
1  [ The ps5 looks like it's gonna be great and h...  
2  [ Well.. Not bad.. \n\nBut now.. I'll keep my ...  


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ps5_0_p_topic_info.csv", index=False)
document_info = topic_model.get_document_info(ps5_0_p['Document_x'])
document_info.to_csv("ps5_0_p_document_info.csv", index=False)

In [ ]:
ps5_1_p = pd.read_csv('/content/ps5_1_p.csv')

In [ ]:
topics, probs = topic_model.fit_transform(ps5_1_p['Document_x'])
print(topic_model.get_topic_info())

   Topic  Count                        Name  \
0     -1     11  -1_the_controller_xbox_you   

                                      Representation  \
0  [the, controller, xbox, you, controllers, and,...   

                                 Representative_Docs  
0  [ i hobestly like the xbox disign more but the...  


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ps5_1_p_topic_info.csv", index=False)
document_info = topic_model.get_document_info(ps5_1_p['Document_x'])
document_info.to_csv("ps5_1_p_document_info.csv", index=False)

In [ ]:
ps5_1_n = pd.read_csv('/content/ps5_1_n.csv')

In [ ]:
topics, probs = topic_model.fit_transform(ps5_1_n['Document_x'])
print(topic_model.get_topic_info())

   Topic  Count                        Name  \
0     -1     24  -1_the_controller_like_and   

                                      Representation  \
0  [the, controller, like, and, is, looks, contro...   

                                 Representative_Docs  
0  [ Personaly i dont like the design from thema ...  


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ps5_1_n_topic_info.csv", index=False)
document_info = topic_model.get_document_info(ps5_1_n['Document_x'])
document_info.to_csv("ps5_1_n_document_info.csv", index=False)

# NS

In [ ]:
ns_launch = pd.read_csv('/content/ns launch.csv')
ns_launch.head()

,url,twitterUrl,id,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote
0,https://x.com/jam71102/status/1817954291119718523,https://twitter.com/jam71102/status/1817954291...,1817954291119718523,@Denariux1 @AdamateurBB @xboxuk @NintendoAmeri...,0,0,0,0,Mon Jul 29 16:04:13 +0000 2024,0,False,False
1,https://x.com/Denariux1/status/181788792524734...,https://twitter.com/Denariux1/status/181788792...,1817887925247340675,@jam71102 @AdamateurBB @xboxuk @NintendoAmeric...,0,1,0,0,Mon Jul 29 11:40:30 +0000 2024,0,False,False
2,https://x.com/funkykongfac/status/172853009628...,https://twitter.com/funkykongfac/status/172853...,1728530096284205148,@LubbaSMG2 @NintendoAmerica good news https://...,0,0,2,0,Sat Nov 25 21:44:24 +0000 2023,0,False,False
3,https://x.com/inkballpro/status/17278483872721...,https://twitter.com/inkballpro/status/17278483...,1727848387272118650,@NintendoAmerica cant wait,0,0,0,0,Fri Nov 24 00:35:32 +0000 2023,0,False,False
4,https://x.com/SimplyAlonso/status/172661807547...,https://twitter.com/SimplyAlonso/status/172661...,1726618075477512345,@NintendoAmerica dude I can't wait,0,0,1,0,Mon Nov 20 15:06:42 +0000 2023,0,False,False


## NMF

In [ ]:
cleaned_ns = clean_text_data(ns_launch, 'text')
cleaned_ns.head()

,url,twitterUrl,id,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote
0,https://x.com/jam71102/status/1817954291119718523,https://twitter.com/jam71102/status/1817954291...,1817954291119718523,i tweeted this months into the switch s life ...,0,0,0,0,Mon Jul 29 16:04:13 +0000 2024,0,False,False
1,https://x.com/Denariux1/status/181788792524734...,https://twitter.com/Denariux1/status/181788792...,1817887925247340675,years later this aged worse than milk,0,1,0,0,Mon Jul 29 11:40:30 +0000 2024,0,False,False
2,https://x.com/funkykongfac/status/172853009628...,https://twitter.com/funkykongfac/status/172853...,1728530096284205148,good news,0,0,2,0,Sat Nov 25 21:44:24 +0000 2023,0,False,False
3,https://x.com/inkballpro/status/17278483872721...,https://twitter.com/inkballpro/status/17278483...,1727848387272118650,cannot wait,0,0,0,0,Fri Nov 24 00:35:32 +0000 2023,0,False,False
4,https://x.com/SimplyAlonso/status/172661807547...,https://twitter.com/SimplyAlonso/status/172661...,1726618075477512345,dude i cannot wait,0,0,1,0,Mon Nov 20 15:06:42 +0000 2023,0,False,False


In [ ]:
processed_ns = preprocess_text(cleaned_ns, 'text')
processed_ns.head()

,url,twitterUrl,id,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote,filtered_text
0,https://x.com/jam71102/status/1817954291119718523,https://twitter.com/jam71102/status/1817954291...,1817954291119718523,i tweeted this months into the switch s life ...,0,0,0,0,Mon Jul 29 16:04:13 +0000 2024,0,False,False,tweet month switch life a age
1,https://x.com/Denariux1/status/181788792524734...,https://twitter.com/Denariux1/status/181788792...,1817887925247340675,years later this aged worse than milk,0,1,0,0,Mon Jul 29 11:40:30 +0000 2024,0,False,False,yearlate age bad milk
2,https://x.com/funkykongfac/status/172853009628...,https://twitter.com/funkykongfac/status/172853...,1728530096284205148,good news,0,0,2,0,Sat Nov 25 21:44:24 +0000 2023,0,False,False,good news
3,https://x.com/inkballpro/status/17278483872721...,https://twitter.com/inkballpro/status/17278483...,1727848387272118650,cannot wait,0,0,0,0,Fri Nov 24 00:35:32 +0000 2023,0,False,False,wait
4,https://x.com/SimplyAlonso/status/172661807547...,https://twitter.com/SimplyAlonso/status/172661...,1726618075477512345,dude i cannot wait,0,0,1,0,Mon Nov 20 15:06:42 +0000 2023,0,False,False,dude wait


In [ ]:
vectorizer = TfidfVectorizer(max_features=10000)
tfidf_matrix = vectorizer.fit_transform(processed_ns['filtered_text'])
nmf_model = NMF(n_components=8, init='nndsvd', random_state=42)
nmf_topics = nmf_model.fit_transform(tfidf_matrix)

for index, topic in enumerate(nmf_model.components_):
    print(f"Top 10 words for topic #{index}")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print("\n")

Top 10 words for topic #0
['march', 'lookincredible', 'ready', 'name', 'cd', 'nintendo', 'lite', 'make', 'bad', 'switch']


Top 10 words for topic #1
['need', 'look', 'good', 'play', 'like', 'well', 'wii', 'go', 'game', 'get']


Top 10 words for topic #2
['holy', 'continue', 'setup', 'solved', 'gimme', 'right', 'need', 'sell', 'take', 'takemoney']


Top 10 words for topic #3
['yes', 'idea', 'excite', 'trailer', 'really', 'nice', 'shuttakemoney', 'guy', 'org', 'love']


Top 10 words for topic #4
['newconsole', 'see', 'wow', 'march', 'system', 'interest', 'getone', 'amaze', 'awesome', 'wait']


Top 10 words for topic #5
['remember', 'train', 'aboard', 'dude', 'finally', 'awesome', 'system', 'let', 'holy', 'hype']


Top 10 words for topic #6
['trailer', 'portable', 'remastered', 'show', 'console', 'come', 'holy', 'play', 'saw', 'syria']


Top 10 words for topic #7
['goodjob', 'lookamaze', 'win', 'back', 'excite', 'amaze', 'want', 'console', 'thank', 'nintendo']




## BERTopic

In [ ]:
# 拟合并转换文档
topics, probs = topic_model.fit_transform(ns_launch['text'])

# 显示主题
print(topic_model.get_topic_info())

    Topic  Count                                              Name  \
0      -1    512                    -1_nintendoamerica_it_its_will   
1       0     80                         0_switch_nintendo_the_for   
2       1     75                        1_looks_amazing_great_look   
3       2     72                2_console_portable_consoles_tablet   
4       3     72                              3_skyrim_was_on_that   
..    ...    ...                                               ...   
78     77      5                 77_ze_cherylktje_starters_gaziibo   
79     78      5                        78_fan_goddamn_pissed_your   
80     79      5  79_copernicusnerd_deltapie7_tune_spectrldynamite   
81     80      5            80_lit_iexudesarcasm_chromebook_cringe   
82     81      5          81_aggregated_praised_beatifull_nanorods   

                                       Representation  \
0   [nintendoamerica, it, its, will, the, in, my, ...   
1   [switch, nintendo, the, for, 2017, is, lo

In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ns_topic_info.csv", index=False)

In [ ]:
document_info = topic_model.get_document_info(ns_launch['text'])
document_info.to_csv("ns_document_info.csv", index=False)

# NS OLED

In [ ]:
ns_oled_launch1 = pd.read_csv('/content/ns oled1.csv')
ns_oled_launch2 = pd.read_csv('/content/ns oled2.csv')
ns_oled_launch = pd.concat([ns_oled_launch1, ns_oled_launch2],ignore_index=True)
ns_oled_launch.head()

,url,twitterUrl,id,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote
0,https://x.com/lunaswagmoney/status/14646111243...,https://twitter.com/lunaswagmoney/status/14646...,1464611124368785409,@shoryufallouts @Anubisty @NintendoAmerica tel...,0,1,1,0,Sat Nov 27 15:04:37 +0000 2021,0,False,False
1,https://x.com/RabTabGames/status/1438029272120...,https://twitter.com/RabTabGames/status/1438029...,1438029272120238084,@BootKuribo @NintendoAmerica 2 Months later.,0,0,0,0,Wed Sep 15 06:37:50 +0000 2021,0,False,False
2,https://x.com/zynx85104590/status/143581619265...,https://twitter.com/zynx85104590/status/143581...,1435816192657211396,@NintendoAmerica 😂 you Gotta be trolling @Nint...,0,0,0,0,Thu Sep 09 04:03:51 +0000 2021,0,False,False
3,https://x.com/Arch0020/status/1435272722574086144,https://twitter.com/Arch0020/status/1435272722...,1435272722574086144,@NintendoAmerica what about the 4k and dont se...,0,0,1,0,Tue Sep 07 16:04:17 +0000 2021,0,False,False
4,https://x.com/Eeve2espeon/status/1418402865748...,https://twitter.com/Eeve2espeon/status/1418402...,1418402865748484100,@Plurlmao @MiiCostumeGuy @NintendoAmerica you ...,0,0,0,0,Fri Jul 23 02:49:30 +0000 2021,0,False,False


In [ ]:
cleaned_ns_oled = clean_text_data(ns_oled_launch, 'text')
cleaned_ns_oled.head()

,url,twitterUrl,id,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote
0,https://x.com/lunaswagmoney/status/14646111243...,https://twitter.com/lunaswagmoney/status/14646...,1464611124368785409,tell them,0,1,1,0,Sat Nov 27 15:04:37 +0000 2021,0,False,False
1,https://x.com/RabTabGames/status/1438029272120...,https://twitter.com/RabTabGames/status/1438029...,1438029272120238084,months later,0,0,0,0,Wed Sep 15 06:37:50 +0000 2021,0,False,False
2,https://x.com/zynx85104590/status/143581619265...,https://twitter.com/zynx85104590/status/143581...,1435816192657211396,you got to be trolling,0,0,0,0,Thu Sep 09 04:03:51 +0000 2021,0,False,False
3,https://x.com/Arch0020/status/1435272722574086144,https://twitter.com/Arch0020/status/1435272722...,1435272722574086144,what about the k and do not send the nintendo...,0,0,1,0,Tue Sep 07 16:04:17 +0000 2021,0,False,False
4,https://x.com/Eeve2espeon/status/1418402865748...,https://twitter.com/Eeve2espeon/status/1418402...,1418402865748484100,you do know playing games like those on a mob...,0,0,0,0,Fri Jul 23 02:49:30 +0000 2021,0,False,False


In [ ]:
processed_ns_oled = preprocess_text(cleaned_ns_oled, 'text')
processed_ns_oled.head()

,url,twitterUrl,id,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote,filtered_text
0,https://x.com/lunaswagmoney/status/14646111243...,https://twitter.com/lunaswagmoney/status/14646...,1464611124368785409,tell them,0,1,1,0,Sat Nov 27 15:04:37 +0000 2021,0,False,False,tell
1,https://x.com/RabTabGames/status/1438029272120...,https://twitter.com/RabTabGames/status/1438029...,1438029272120238084,months later,0,0,0,0,Wed Sep 15 06:37:50 +0000 2021,0,False,False,month late
2,https://x.com/zynx85104590/status/143581619265...,https://twitter.com/zynx85104590/status/143581...,1435816192657211396,you got to be trolling,0,0,0,0,Thu Sep 09 04:03:51 +0000 2021,0,False,False,get troll
3,https://x.com/Arch0020/status/1435272722574086144,https://twitter.com/Arch0020/status/1435272722...,1435272722574086144,what about the k and do not send the nintendo...,0,0,1,0,Tue Sep 07 16:04:17 +0000 2021,0,False,False,a send nintendo ninja would pay upgrade compon...
4,https://x.com/Eeve2espeon/status/1418402865748...,https://twitter.com/Eeve2espeon/status/1418402...,1418402865748484100,you do know playing games like those on a mob...,0,0,0,0,Fri Jul 23 02:49:30 +0000 2021,0,False,False,know play game like mobiledevice also require ...


In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = vectorizer.fit_transform(processed_ns_oled['filtered_text'])
nmf_model = NMF(n_components=6, init='nndsvd', random_state=42)
nmf_topics = nmf_model.fit_transform(tfidf_matrix)

for index, topic in enumerate(nmf_model.components_):
    print(f"Top 10 words for topic #{index}")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print("\n")

Top 10 words for topic #0
['to', 'like', 'dock', 'still', 'doe', 'want', 'would', 'play', 'well', 'game']


Top 10 words for topic #1
['model', 'launch', 'year', 'upgrade', 'dock', 'already', 'of', 'old', 'new', 'switch']


Top 10 words for topic #2
['reason', 'want', 'doe', 'another', 'normalswitch', 'anything', 'people', 'already', 'one', 'buy']


Top 10 words for topic #3
['model', 'mad', 'probably', 'point', 'already', 'dock', 'one', 'none', 'lao', 'get']


Top 10 words for topic #4
['never', 'year', 'fan', 'love', 'think', 'switchpro', 'like', 'old', 'know', 'nintendo']


Top 10 words for topic #5
['methoddread', 'say', 'storage', 'play', 'look', 'of', 'tell', 'low', 'like', 'go']




## BERTopic

In [ ]:
# 拟合并转换文档
topics, probs = topic_model.fit_transform(ns_oled_launch['text'])

# 显示主题
print(topic_model.get_topic_info())

    Topic  Count                                               Name  \
0      -1    730                     -1_nintendoamerica_the_to_this   
1       0    211                               0_switch_new_the_one   
2       1    132                               1_oled_screen_the_is   
3       2     82    2_among_nintendoamerica_croppingzero_earthbound   
4       3     67  3_free_alexglasscock_nintendoamerica_jordankad...   
..    ...    ...                                                ...   
85     84      5               84_doespichu_cum_copperlockscum_your   
86     85      5                    85_forbut_hearing_wallet_showed   
87     86      5                             86_dock_work_older_3mm   
88     87      5                  87_nintendos_rumors_trouble_poopy   
89     88      5                 88_acecourtbot_render_judge_jammin   

                                       Representation  \
0   [nintendoamerica, the, to, this, and, you, it,...   
1   [switch, new, the, one, have,

In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ns_oled_topic_info.csv", index=False)

In [ ]:
document_info = topic_model.get_document_info(ns_oled_launch['text'])
document_info.to_csv("ns_oled_document_info.csv", index=False)

# NS2

In [ ]:
ns2_launch = pd.read_csv('/content/ns2.csv')
ns2_launch.head()

,url,twitterUrl,id,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote
0,https://x.com/1888Jaar/status/1798313334611341398,https://twitter.com/1888Jaar/status/1798313334...,1798313334611341398,@MarilouBurnel @NinSwitchIntel What for?,0,0,0,0,Wed Jun 05 11:18:04 +0000 2024,0,False,False
1,https://x.com/fros7696_/status/178130313191930...,https://twitter.com/fros7696_/status/178130313...,1781303131919302835,@NinSwitchIntel The memory is 128GB and the de...,0,0,0,0,Fri Apr 19 12:45:36 +0000 2024,0,False,False
2,https://x.com/johnny_univo/status/175813706188...,https://twitter.com/johnny_univo/status/175813...,1758137061885042726,@NinSwitchIntel @ShucaybKarie Simply backwards...,1,0,1,0,Thu Feb 15 14:31:54 +0000 2024,0,False,False
3,https://x.com/Jordan456015Lee/status/175567476...,https://twitter.com/Jordan456015Lee/status/175...,1755674760388788248,@NinSwitchIntel Bigger memory so it can play x...,0,0,0,0,Thu Feb 08 19:27:36 +0000 2024,0,False,False
4,https://x.com/PepitoKun5/status/17514042310540...,https://twitter.com/PepitoKun5/status/17514042...,1751404231054008634,@NinSwitchIntel Spotify,0,0,1,0,Sun Jan 28 00:38:02 +0000 2024,0,False,False


In [ ]:
cleaned_ns2 = clean_text_data(ns2_launch, 'text')
cleaned_ns2.head()

,url,twitterUrl,id,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote
0,https://x.com/1888Jaar/status/1798313334611341398,https://twitter.com/1888Jaar/status/1798313334...,1798313334611341398,what for,0,0,0,0,Wed Jun 05 11:18:04 +0000 2024,0,False,False
1,https://x.com/fros7696_/status/178130313191930...,https://twitter.com/fros7696_/status/178130313...,1781303131919302835,the memory is gb and the device has developed...,0,0,0,0,Fri Apr 19 12:45:36 +0000 2024,0,False,False
2,https://x.com/johnny_univo/status/175813706188...,https://twitter.com/johnny_univo/status/175813...,1758137061885042726,simply backwards compatibility,1,0,1,0,Thu Feb 15 14:31:54 +0000 2024,0,False,False
3,https://x.com/Jordan456015Lee/status/175567476...,https://twitter.com/Jordan456015Lee/status/175...,1755674760388788248,bigger memory so it can play xbox or playstat...,0,0,0,0,Thu Feb 08 19:27:36 +0000 2024,0,False,False
4,https://x.com/PepitoKun5/status/17514042310540...,https://twitter.com/PepitoKun5/status/17514042...,1751404231054008634,spotify,0,0,1,0,Sun Jan 28 00:38:02 +0000 2024,0,False,False


In [ ]:
processed_ns2 = preprocess_text(cleaned_ns2, 'text')
processed_ns2.head()

,url,twitterUrl,id,text,retweetCount,replyCount,likeCount,quoteCount,createdAt,bookmarkCount,isRetweet,isQuote,filtered_text
0,https://x.com/1888Jaar/status/1798313334611341398,https://twitter.com/1888Jaar/status/1798313334...,1798313334611341398,what for,0,0,0,0,Wed Jun 05 11:18:04 +0000 2024,0,False,False,
1,https://x.com/fros7696_/status/178130313191930...,https://twitter.com/fros7696_/status/178130313...,1781303131919302835,the memory is gb and the device has developed...,0,0,0,0,Fri Apr 19 12:45:36 +0000 2024,0,False,False,memory go device a develop internal specification
2,https://x.com/johnny_univo/status/175813706188...,https://twitter.com/johnny_univo/status/175813...,1758137061885042726,simply backwards compatibility,1,0,1,0,Thu Feb 15 14:31:54 +0000 2024,0,False,False,simply backwardscompatibility
3,https://x.com/Jordan456015Lee/status/175567476...,https://twitter.com/Jordan456015Lee/status/175...,1755674760388788248,bigger memory so it can play xbox or playstat...,0,0,0,0,Thu Feb 08 19:27:36 +0000 2024,0,False,False,big memory play box playstation platform game
4,https://x.com/PepitoKun5/status/17514042310540...,https://twitter.com/PepitoKun5/status/17514042...,1751404231054008634,spotify,0,0,1,0,Sun Jan 28 00:38:02 +0000 2024,0,False,False,notify


In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = vectorizer.fit_transform(processed_ns2['filtered_text'])
nmf_model = NMF(n_components=6, init='nndsvd', random_state=42)
nmf_topics = nmf_model.fit_transform(tfidf_matrix)

for index, topic in enumerate(nmf_model.components_):
    print(f"Top 10 words for topic #{index}")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print("\n")

Top 10 words for topic #0
['could', 'make', 'pro', 'powerful', 'work', 'game', 'call', 'would', 'nintendo', 'switch']


Top 10 words for topic #1
['much', 'cartridge', 'instead', 'menu', 'bpm', 'must', 'custom', 'please', 'change', 'theme']


Top 10 words for topic #2
['resolution', 'footnote', 'reliable', 'make', 'one', 'graphic', 'procontroller', 'controller', 'concept', 'well']


Top 10 words for topic #3
['streetcars', 'hardware', 'care', 'cartridge', 'simply', 'pray', 'physical', 'game', 'new', 'backwardscompatibility']


Top 10 words for topic #4
['playstation', 'support', 'platform', 'good', 'internetbrowser', 'play', 'achievement', 'game', 'wii', 'like']


Top 10 words for topic #5
['wouldnice', 'run', 'pokemon', 'brut', 'game', 'dock', 'norm', 'everything', 'least', 'fps']




## BERTopic

In [ ]:
# 拟合并转换文档
topics, probs = topic_model.fit_transform(ns2_launch['text'])

# 显示主题
print(topic_model.get_topic_info())

    Topic  Count                                           Name  \
0      -1     55                   -1_ninswitchintel_to_the_and   
1       0     43            0_ninswitchintel_for_yoshiestela_to   
2       1     20                         1_games_and_for_better   
3       2     18  2_compatibility_backwards_ninswitchintel_else   
4       3     18                       3_switch_the_nintendo_it   
5       4     15       4_themes_ninswitchintel_menus_background   
6       5     13                     5_60_fps_ninswitchintel_30   
7       6     12                6_better_controller_pro_concept   
8       7     12        7_wii_console_marilouburnel_somejudge99   
9       8      9                       8_music_eshop_themes_the   
10      9      6                    9_streetpass_in_feature_was   

                                       Representation  \
0   [ninswitchintel, to, the, and, switch, be, hav...   
1   [ninswitchintel, for, yoshiestela, to, drift, ...   
2   [games, and, for, be

In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.to_csv("ns2_topic_info.csv", index=False)

In [ ]:
document_info = topic_model.get_document_info(ns2_launch['text'])
document_info.to_csv("ns2_document_info.csv", index=False)